In [1]:
##### import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST
import os
import torch.nn.functional as F
import cv2
import numpy as np
import torch.utils.data as Data
from PIL import  Image,ImageDraw
import numpy as np
import random
import torch 

Batch_Size = 50
num_epochs = 200
learning_rate = 1e-5
samples = 20000

data1 = []
data2= []


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% --  Generate dataset


cap = cv2.VideoCapture('video_drone_tank.avi')

uframes = []
ret, frame = cap.read()

uframes.append(frame)
flag = 0
z = 0
while(z<30):
    ret, frame = cap.read()
    #np_frame = cv2.imread('video', frame) # does not work
    #np_frame = np.asarray(cv2.GetMat(frame)) # does not work
    #print(np_frame.shape)
    for a in uframes:
        if (np.array(a).any() != None):
            print("a: ",np.shape(a))
            print("frame: ",np.shape(a))
            difference = cv2.subtract(np.array(a), np.array(frame))    
            result = not np.any(difference)
            #print("inside for")
            if result is True:
                flag = 1
                #print("setting flag")
                break
    if(not flag):
        #print("appending")
        uframes.append(frame)
    z+=1

for i,frame in enumerate(uframes):
    uframes[i] = cv2.resize(frame,(100,100))
    
print(np.shape(uframes[0]))
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()
print("length: ",len(uframes))
for i in range(samples):
    data1.append(np.array(np.transpose(np.asarray(uframes[i % len(uframes)]),(2,0,1)),dtype=np.float32))
    data2.append(np.array(np.transpose(np.asarray(uframes[(i+1) % len(uframes)]),(2,0,1)),dtype=np.float32))

    

#%%%%%%%%%%%%%%%%%%%%%%%%%%% Noisy data

im1 = Image.open('tree1.png')
im1 = im1.resize((10,10))

im2 = Image.open('tree2.png')
im2 = im2.resize((10,10))

im3 = Image.open('tree3.png')
im3 = im3.resize((10,10))

img = Image.new('RGB', (100,100), color = '#9b7653')
img.paste(im1,(5,75),im1)
img.paste(im1,(10,55),im1)
img.paste(im1,(25,50),im1)
img.paste(im2,(75,5),im2)
img.paste(im2,(60,30),im2)
img.paste(im2,(90,20),im2)
img.paste(im3,(75,60),im3)
img.paste(im3,(60,80),im3)
im_n = np.reshape(img,(100,100,3))
b = np.asarray(im_n)
b = np.array(b, dtype=np.float32)
b = np.transpose(b,(2,0,1))
b = torch.from_numpy(b)

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%       

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% - Data Preparation

y0 = np.asarray(data1)
y1 = np.asarray(data2)



print(np.shape(y0))

x = torch.from_numpy(y0)
y = torch.from_numpy(y1)


torch_dataset = Data.TensorDataset(x,y)


loader = Data.DataLoader(

    dataset=torch_dataset,

    batch_size=Batch_Size,

    shuffle=True,

    num_workers=0,

)


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% - Defining NN

class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()

        # Convolution 1
        self.cnn1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5, stride=1, padding=2)
        self.relu1 = nn.ReLU()

        # Max pool 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)

        # Convolution 2
        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.relu2 = nn.ReLU()

        # Max pool 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)

        # Fully connected 1 (readout)
        self.fc1 = nn.Linear(20000, 3)
        self.sigmoid1 = nn.Sigmoid()

    def forward(self, x):
        # Convolution 1
        out = self.cnn1(x)
        out = self.relu1(out)

        # Max pool 1
        out = self.maxpool1(out)

        # Convolution 2 
        out = self.cnn2(out)
        out = self.relu2(out)

        # Max pool 2 
        out = self.maxpool2(out)

        # Resize
        # Original size: (100, 32, 7, 7)
        # out.size(0): 100
        # New out size: (100, 32*7*7)
        out = out.view(out.size(0), -1)

        # Linear function (readout)
        out = self.fc1(out)
        #out = self.sigmoid1(out)
        return out
    
model = CNNModel()


optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% - Loss Function

def h_score(fx, gy):
 
    fx = fx - fx.mean(0)

    gy = gy - gy.mean(0)

    Nsamples = fx.size(0)
    #print(fx, np.shape(fx))
    #print(gy, np.shape(gy))
    #print(np.shape((torch.transpose(fx,3,2))),np.shape(fx))
    covf = torch.matmul(fx.t(), fx) / Nsamples

    covg = torch.matmul(gy.t(), gy) / Nsamples

    h = -2 * torch.mean((fx * gy).sum(1)) + (covf * covg).sum()

    return h


optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay=1e-4)





for epoch in range(num_epochs):
    for x,y in loader:
        optimizer.zero_grad()
        # ===================forward=====================
        #loss = criterion(output1, img)
        loss = h_score(model(x),model(y))
        # ===================backward====================

        #optimizer_1.zero_grad()
        loss.backward()
        optimizer.step()
        #optimizer_1.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'.format(epoch+1, num_epochs, loss.item()))
    print("x: ", model(x[0].reshape(1,3,100, 100)))
    print("y:", model(y[0].reshape(1,3,100, 100)))
    print("b:", model(b.reshape(1,3,100, 100)))


a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 

epoch [5/200], loss:-1.1121
x:  tensor([[ 37.1090, -74.7536, -27.9565]], grad_fn=<AddmmBackward>)
y: tensor([[ 36.7237, -75.1770, -27.4767]], grad_fn=<AddmmBackward>)
b: tensor([[ 30.2548, -67.5620, -21.9423]], grad_fn=<AddmmBackward>)
epoch [6/200], loss:-1.3582
x:  tensor([[ 57.0620, -74.2956, -42.2361]], grad_fn=<AddmmBackward>)
y: tensor([[ 57.4888, -74.6400, -40.6732]], grad_fn=<AddmmBackward>)
b: tensor([[ 48.1848, -67.5606, -34.6384]], grad_fn=<AddmmBackward>)
epoch [7/200], loss:-1.4796
x:  tensor([[ 50.7582, -66.3222, -52.7143]], grad_fn=<AddmmBackward>)
y: tensor([[ 51.5050, -66.4383, -53.1832]], grad_fn=<AddmmBackward>)
b: tensor([[ 42.7401, -60.8278, -44.7940]], grad_fn=<AddmmBackward>)
epoch [8/200], loss:-1.3141
x:  tensor([[ 54.4459, -63.9226, -20.1232]], grad_fn=<AddmmBackward>)
y: tensor([[ 54.1264, -64.4595, -19.7013]], grad_fn=<AddmmBackward>)
b: tensor([[ 45.1484, -59.3492, -16.1437]], grad_fn=<AddmmBackward>)
epoch [9/200], loss:-0.9943
x:  tensor([[110.2284, -62.1

y: tensor([[-7.6980, 48.0061, 12.4019]], grad_fn=<AddmmBackward>)
b: tensor([[-7.6000, 38.8688, 11.9666]], grad_fn=<AddmmBackward>)
epoch [41/200], loss:-1.4452
x:  tensor([[2.5713, 6.1713, 0.4162]], grad_fn=<AddmmBackward>)
y: tensor([[2.3282, 5.0565, 0.4086]], grad_fn=<AddmmBackward>)
b: tensor([[0.1503, 3.2574, 1.3617]], grad_fn=<AddmmBackward>)
epoch [42/200], loss:-1.5305
x:  tensor([[-11.5046, -18.1353,   0.6244]], grad_fn=<AddmmBackward>)
y: tensor([[-10.8938, -17.8404,   0.3535]], grad_fn=<AddmmBackward>)
b: tensor([[ -9.9814, -16.2282,   2.4072]], grad_fn=<AddmmBackward>)
epoch [43/200], loss:-1.4543
x:  tensor([[18.2975, 16.6447, 31.3431]], grad_fn=<AddmmBackward>)
y: tensor([[17.9565, 15.7982, 31.4493]], grad_fn=<AddmmBackward>)
b: tensor([[13.7191, 10.8553, 27.6332]], grad_fn=<AddmmBackward>)
epoch [44/200], loss:-1.5745
x:  tensor([[27.0008, 18.7323, 23.3790]], grad_fn=<AddmmBackward>)
y: tensor([[27.3135, 19.1478, 24.3520]], grad_fn=<AddmmBackward>)
b: tensor([[19.5386, 1

epoch [76/200], loss:-1.3666
x:  tensor([[ 49.2808,   1.1898, -15.6632]], grad_fn=<AddmmBackward>)
y: tensor([[ 49.6149,   1.5764, -14.6863]], grad_fn=<AddmmBackward>)
b: tensor([[ 37.2354,   0.9434, -10.4654]], grad_fn=<AddmmBackward>)
epoch [77/200], loss:-1.4955
x:  tensor([[ 21.3602, -14.2205, -18.4246]], grad_fn=<AddmmBackward>)
y: tensor([[ 20.0352, -13.4056, -18.6738]], grad_fn=<AddmmBackward>)
b: tensor([[ 14.5382, -11.9089, -13.8503]], grad_fn=<AddmmBackward>)
epoch [78/200], loss:-1.4535
x:  tensor([[ 19.4768, -17.4088, -34.5249]], grad_fn=<AddmmBackward>)
y: tensor([[ 20.4271, -16.7352, -34.8965]], grad_fn=<AddmmBackward>)
b: tensor([[ 14.3990, -14.4016, -25.9448]], grad_fn=<AddmmBackward>)
epoch [79/200], loss:-1.2832
x:  tensor([[ 32.4311, -30.4910, -15.4943]], grad_fn=<AddmmBackward>)
y: tensor([[ 32.1821, -31.4630, -15.4890]], grad_fn=<AddmmBackward>)
b: tensor([[ 24.3361, -25.7447, -10.9842]], grad_fn=<AddmmBackward>)
epoch [80/200], loss:-1.4039
x:  tensor([[ 13.1817, 

y: tensor([[ 12.1201,   6.0814, -13.6861]], grad_fn=<AddmmBackward>)
b: tensor([[ 5.5373,  3.0470, -7.1766]], grad_fn=<AddmmBackward>)
epoch [112/200], loss:-1.2879
x:  tensor([[11.9970, -6.8175, -1.4816]], grad_fn=<AddmmBackward>)
y: tensor([[11.3527, -7.1243, -2.3341]], grad_fn=<AddmmBackward>)
b: tensor([[ 7.0536, -6.2615,  0.0115]], grad_fn=<AddmmBackward>)
epoch [113/200], loss:-1.5269
x:  tensor([[ -5.6662, -16.6197,   8.3577]], grad_fn=<AddmmBackward>)
y: tensor([[ -5.0228, -16.2823,   7.9687]], grad_fn=<AddmmBackward>)
b: tensor([[ -6.0778, -12.9195,   8.2720]], grad_fn=<AddmmBackward>)
epoch [114/200], loss:-1.5040
x:  tensor([[5.0198, 2.9883, 6.4254]], grad_fn=<AddmmBackward>)
y: tensor([[4.8801, 2.0508, 6.4516]], grad_fn=<AddmmBackward>)
b: tensor([[1.1032, 0.5476, 6.2480]], grad_fn=<AddmmBackward>)
epoch [115/200], loss:-1.3084
x:  tensor([[24.4841,  3.2065,  0.4662]], grad_fn=<AddmmBackward>)
y: tensor([[25.1646,  3.6354,  0.0932]], grad_fn=<AddmmBackward>)
b: tensor([[16.

epoch [147/200], loss:-1.5205
x:  tensor([[22.2446,  1.3896, 29.1143]], grad_fn=<AddmmBackward>)
y: tensor([[21.7817,  2.3755, 29.5519]], grad_fn=<AddmmBackward>)
b: tensor([[12.6291,  1.7031, 23.3994]], grad_fn=<AddmmBackward>)
epoch [148/200], loss:-1.2993
x:  tensor([[14.1096, 12.2825,  8.8321]], grad_fn=<AddmmBackward>)
y: tensor([[14.8289, 11.6487,  9.7306]], grad_fn=<AddmmBackward>)
b: tensor([[ 7.5625,  9.2555, 10.0746]], grad_fn=<AddmmBackward>)
epoch [149/200], loss:-1.4613
x:  tensor([[23.5859,  0.1542, 23.9360]], grad_fn=<AddmmBackward>)
y: tensor([[24.3110, -0.4915, 24.9438]], grad_fn=<AddmmBackward>)
b: tensor([[14.4570,  0.4775, 21.0602]], grad_fn=<AddmmBackward>)
epoch [150/200], loss:-1.4367
x:  tensor([[ 7.3306, -5.4787, -1.1015]], grad_fn=<AddmmBackward>)
y: tensor([[ 8.1271, -6.1515,  0.0084]], grad_fn=<AddmmBackward>)
b: tensor([[ 2.9934, -3.8370,  3.5670]], grad_fn=<AddmmBackward>)
epoch [151/200], loss:-1.3456
x:  tensor([[ 2.7984, -1.6611, -8.8137]], grad_fn=<Add

y: tensor([[11.6718, -8.5303, -6.2438]], grad_fn=<AddmmBackward>)
b: tensor([[ 4.5031, -6.7043, -1.2414]], grad_fn=<AddmmBackward>)
epoch [183/200], loss:-1.4247
x:  tensor([[ -6.5772, -19.1412,  -6.7325]], grad_fn=<AddmmBackward>)
y: tensor([[ -6.7460, -20.0606,  -6.7106]], grad_fn=<AddmmBackward>)
b: tensor([[ -8.0890, -14.1101,  -1.4204]], grad_fn=<AddmmBackward>)
epoch [184/200], loss:-1.3065
x:  tensor([[-0.1479, -6.3139, -5.9541]], grad_fn=<AddmmBackward>)
y: tensor([[ 0.7889, -5.5154, -6.2029]], grad_fn=<AddmmBackward>)
b: tensor([[-3.0545, -4.8272, -1.3245]], grad_fn=<AddmmBackward>)
epoch [185/200], loss:-1.5003
x:  tensor([[-1.8234,  8.6735,  9.5881]], grad_fn=<AddmmBackward>)
y: tensor([[-1.0019,  9.4322,  9.2157]], grad_fn=<AddmmBackward>)
b: tensor([[-3.8709,  6.1048,  9.2816]], grad_fn=<AddmmBackward>)
epoch [186/200], loss:-1.3948
x:  tensor([[ 7.1244, -8.2860, -7.7475]], grad_fn=<AddmmBackward>)
y: tensor([[ 7.3944, -7.7363, -6.8487]], grad_fn=<AddmmBackward>)
b: tensor

In [3]:
data3=[]

class_data = []

for i in range(75):
    img = uframes[0]
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(40,40),im_drone)
    #img.paste(imb,(80,80),imb)
    #draw = ImageDraw.Draw(img)
    #draw.ellipse((50,50,55,55), fill=(0, 0, 255))
    #img.show()
    #im2_pos1 = np.reshape(im2_pos1,(100,100,3))
    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    class_data.append((list(temp.detach().numpy()[0]),0))



In [7]:
data3=[]

for i in range(75):
    img = uframes[1]

    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    class_data.append((list(temp.detach().numpy()[0]),2))
    
print(len(class_data))

NameError: name 'class_data' is not defined

In [5]:
data3=[]

for i in range(75):
    img = uframes[2]
    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    class_data.append((list(temp.detach().numpy()[0]),3))
    
print(len(class_data))

225


In [8]:
import random


p=0
class_data = []
while(p<len(uframes)):
    data3=[]
    print(p)
    for i in range(50):
        img = uframes[p]
        data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
        y3 = np.array(data3)
        x_1 = torch.from_numpy(y3)
        temp = model(x_1[i].reshape(1,3,100,100))
        #print(temp)
        class_data.append((list(temp.detach().numpy()[0]),p))
    p+=1
    print(len(class_data))

random.shuffle(class_data)
random.shuffle(class_data)

print("length ", len(class_data))

0
50
1
100
2
150
3
200
4
250
5
300
6
350
7
400
8
450
9
500
10
550
11
600
length  600


In [6]:
data3=[]

for i in range(75):
    img = uframes[3]

    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    class_data.append((list(temp.detach().numpy()[0]),1))
    
print(len(class_data))

300


In [18]:

cap = cv2.VideoCapture('video_drone_tank.avi')

uframes = []
ret, frame = cap.read()

uframes.append(frame)
flag = 0
z = 0
while(z<30):
    ret, frame = cap.read()
    #np_frame = cv2.imread('video', frame) # does not work
    #np_frame = np.asarray(cv2.GetMat(frame)) # does not work
    #print(np_frame.shape)
    for a in uframes:
        if (np.array(a).any() != None):
            print("a: ",np.shape(a))
            print("frame: ",np.shape(a))
            difference = cv2.subtract(np.array(a), np.array(frame))    
            result = not np.any(difference)
            #print("inside for")
            if result is True:
                flag = 1
                #print("setting flag")
                break
    if(not flag):
        #print("appending")
        uframes.append(frame)
    z+=1

frame = Image.fromarray(uframes[0])


video = cv2.VideoWriter('video_test.avi', 0, 1, (720,1080))

for image in uframes:
    video.write(image)


for i,frame in enumerate(uframes):
    uframes[i] = cv2.resize(frame,(100,100))
cv2.destroyAllWindows()
video.release()

a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 

In [21]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from colorama import init 
from termcolor import colored 
import time
print(np.shape(uframes[0]))


#z  = np.array(np.transpose(np.asarray(z),(2,0,1)),dtype=np.float32)
# import some data to play with
print("Length of Class data: ", len(class_data))
# Take the first two features. We could avoid this by using a two-dim dataset
X = [(i[0]) for i in class_data]
y = [i[1] for i in class_data]
#print(X,np.shape(X))
#print(y,np.shape(y))
# we create an instance of SVM and fit out data. We do not scale our
# data since we want to plot the support vectors
C = 1.0  # SVM regularization parameter
clf = svm.SVC(kernel='linear', C=C)
clf = clf.fit(X, y)
data3=[]
y3=0

n1 = "\x1b[31m\"Drone Stationary, Tank Firing\"\x1b[0m"
n2 = "\x1b[31m\"Drone Reloading, Tank Firing\"\x1b[0m"
n3 = "\x1b[31m\"Drone Firing, Tank Stationary\"\x1b[0m"
n4 = "\x1b[31m\"Drone Firing, Tank Reloading\"\x1b[0m"
n5 = "\x1b[31m\"Drone Firing, Tank Firing\"\x1b[0m"

names = ['Just Background','Only Tank moving', 'Only Drone Moving', 'Drone stationary, Tank Stationary', 'Drone Stationary, Tank Reloading' ,
         n1, 'Drone reloading, Tank Stationary', 'Drone reloading, Tank Reloading', n2 , n3,n4,n5]

print("Start")
for i in range(12):
    z = cv2.imread('Images1/0i.png')
    z  = cv2.resize(z,(100,100))
    img = uframes[i]
    print(np.shape(img))
    #ms = Image.fromarray(z)
    #ms.show()
    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))
    temp = temp.detach().numpy()
    #print(temp)
    z = clf.predict(temp)
    print(names[z[0]])
    


(100, 100, 3)
Length of Class data:  600
Start
(100, 100, 3)
Just Background
(100, 100, 3)
Only Tank moving
(100, 100, 3)
Only Drone Moving
(100, 100, 3)
Drone stationary, Tank Stationary
(100, 100, 3)
Drone Stationary, Tank Reloading
(100, 100, 3)
"Drone Stationary, Tank Firing"
(100, 100, 3)
Drone reloading, Tank Stationary
(100, 100, 3)
Drone reloading, Tank Reloading
(100, 100, 3)
"Drone Reloading, Tank Firing"
(100, 100, 3)
"Drone Firing, Tank Stationary"
(100, 100, 3)
"Drone Firing, Tank Reloading"
(100, 100, 3)
"Drone Firing, Tank Firing"


In [54]:
data3=[]
for i in range(20):
    img = Image.new('RGB', (100, 100), color = '#9b7653')
    img.paste(im1,(5,75),im1)
    img.paste(im1,(10,55),im1)
    img.paste(im1,(25,50),im1)
    img.paste(im2,(75,5),im2)
    img.paste(im2,(60,30),im2)
    img.paste(im2,(90,20),im2)
    img.paste(im3,(75,60),im3)
    img.paste(im3,(60,80),im3)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(40,40),im_drone)
    #img.paste(imb,(80,80),imb)
    #draw = ImageDraw.Draw(img)
    #draw.ellipse((50,50,55,55), fill=(0, 0, 255))
    #img.show()
    #im2_pos1 = np.reshape(im2_pos1,(100,100,3))
    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    temp = temp.detach().numpy()
   
    print(temp)
    z = clf.predict(temp)
    print(z)

[[ -1.5474473   3.4351084   5.324524   24.159393    4.10358     6.0005293
   -0.6879165  -2.862772   -5.096247    4.1649837  -2.2589116  -1.0829961
  -10.358628   -9.089189    4.2659616]]
[5]
[[ -1.5474473   3.4351084   5.324524   24.159393    4.10358     6.0005293
   -0.6879165  -2.862772   -5.096247    4.1649837  -2.2589116  -1.0829961
  -10.358628   -9.089189    4.2659616]]
[5]
[[ -1.5474473   3.4351084   5.324524   24.159393    4.10358     6.0005293
   -0.6879165  -2.862772   -5.096247    4.1649837  -2.2589116  -1.0829961
  -10.358628   -9.089189    4.2659616]]
[5]
[[ -1.5474473   3.4351084   5.324524   24.159393    4.10358     6.0005293
   -0.6879165  -2.862772   -5.096247    4.1649837  -2.2589116  -1.0829961
  -10.358628   -9.089189    4.2659616]]
[5]
[[ -1.5474473   3.4351084   5.324524   24.159393    4.10358     6.0005293
   -0.6879165  -2.862772   -5.096247    4.1649837  -2.2589116  -1.0829961
  -10.358628   -9.089189    4.2659616]]
[5]
[[ -1.5474473   3.4351084   5.324524   2

In [55]:
data3=[]
for i in range(20):
    img = Image.new('RGB', (100, 100), color = '#9b7653')
    img.paste(im1,(5,75),im1)
    img.paste(im1,(10,55),im1)
    img.paste(im1,(25,50),im1)
    img.paste(im2,(75,5),im2)
    img.paste(im2,(60,30),im2)
    img.paste(im2,(90,20),im2)
    img.paste(im3,(75,60),im3)
    img.paste(im3,(60,80),im3)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(40,40),im_drone)
    #img.paste(imb,(80,80),imb)
    img.paste(im_drone,(5,5),im_drone)
    #img.show()
    #im2_pos1 = np.reshape(im2_pos1,(100,100,3))

    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    temp = temp.detach().numpy()
   
    print(temp)
    z = clf.predict(temp)
    print(z)

NameError: name 'im_drone' is not defined

In [53]:
data3=[]
for i in range(20):
    img = Image.new('RGB', (100, 100), color = '#9b7653')
    img.paste(im1,(5,75),im1)
    img.paste(im1,(10,55),im1)
    img.paste(im1,(25,50),im1)
    img.paste(im2,(75,5),im2)
    img.paste(im2,(60,30),im2)
    img.paste(im2,(90,20),im2)
    img.paste(im3,(75,60),im3)
    img.paste(im3,(60,80),im3)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(40,40),im_drone)
    #img.paste(imb,(80,80),imb)
    img.paste(im_drone,(72,65),im_drone)
    img.paste(imb,(80,80),imb)
    #img.show()
    #im2_pos1 = np.reshape(im2_pos1,(100,100,3))

    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    temp = temp.detach().numpy()
   
    print(temp)
    z = clf.predict(temp)
    print(z)

NameError: name 'im_drone' is not defined

In [2]:
img = np.transpose(data1[5],(1,2,0))
print(np.shape(np.transpose(data1[5],(1,2,0))))
cv2.imshow('test', img)

img = data2[0]
ms = Image.fromarray(img)
ms.show()

(100, 100, 3)


TypeError: Cannot handle this data type: (1, 1, 100), <f4